In [2]:
# !pip install haversine

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [2]:
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from haversine import haversine

## data load

In [4]:
# 물놀이 장소 데이터
wp_df = pd.read_csv('../../data/물놀이/WP.csv', encoding='utf-8')

# 수유실 및 어린이 이용가능 공중화장실 데이터
toilet_df = pd.read_csv('../../data/물놀이/Toilet.csv', encoding='utf-8')

# 생활편의시설
conv_df = pd.read_csv('../../data/물놀이/Conv.csv', encoding='utf-8')

# 의료및의약시설 데이터
medi_df = pd.read_csv('../../data/물놀이/Medi.csv', encoding='utf-8')

# 안전시설_경찰및소방 데이터
safe112_df = pd.read_csv('../../data/물놀이/Safe112.csv', encoding='utf-8')

# 물놀이_서울시 소방서,안전센터,구조대 위치정보 데이터
safe119_df = pd.read_csv('../../data/물놀이/Safe119.csv', encoding='utf-8')

# 전국주차장 데이터
parking_df = pd.read_csv('../../data/물놀이/Parking.csv', encoding='utf-8')

## 변수 세팅

In [5]:
# 서울 남쪽
# 37.4357971,127.0143777
a = (37.4357971, 127.0143777)  #Latitude, Longitude
b = (37.4357971, 127.0143897)
# 거리 계산
haversine(a, b, unit = 'm')
# 서울시 남쪽에서 경도 0.000012 당 약 1.06m 차이가 존재한다.

a = (37.4357881, 127.0143777)  #Latitude, Longitude
b = (37.4357971, 127.0143777)
# 거리 계산
haversine(a, b, unit = 'm')
# 서울시 남쪽에서 위도 0.000009 당 약 1m 차이가 존재한다.


# 서울 북쪽
# 37.7023592,127.0532626
a = (37.7023592, 127.0532626)  #Latitude, Longitude
b = (37.7023592, 127.0532746)
# 거리 계산
haversine(a, b, unit = 'm')
# 서울시 북쪽에서 경도 0.000012 당 약 1.06m 차이가 존재한다.

a = (37.7023492, 127.0532626)  #Latitude, Longitude
b = (37.7023582, 127.0532626)
# 거리 계산
haversine(a, b, unit = 'm')
# 서울시 북쪽에서 위도 0.000009 당 약 1m 차이가 존재한다.

1.0595134139525815

1.0007557218148433

1.0557283707087306

1.0007557225221673

In [6]:
# 위도 경도 1M에 대한 단위
LO1M = 0.000009
LA1M = 0.000012
# 필터링 기준 거리
TOILET_MAX_RANGE = 200
CONV_MAX_RANGE = 600
DRUG_MAX_RANGE = 600
MEDI_MAX_RANGE = 1000
SAFE112_MAX_RANGE = 1200
SAFE119_MAX_RANGE = 1500
PARKING_MAX_RANGE = 600

In [17]:
faci_dict = {
    'toilet' : [TOILET_MAX_RANGE, 'toiletid', 'toiletla', 'toiletlo', 'toilettype'], 
    'conv' : [CONV_MAX_RANGE, 'convid', 'convla', 'convlo', 'convtype'], 
    'drug' : [DRUG_MAX_RANGE, 'convid', 'convla', 'convlo', 'convtype'], 
    'medi' : [MEDI_MAX_RANGE, 'mediid', 'medila', 'medilo', 'meditype'], 
    'safe112' : [SAFE112_MAX_RANGE, 'safe112id', 'safe112la', 'safe112lo', 'safe112subtype'], 
    'safe119' : [SAFE119_MAX_RANGE, 'safe119id', 'safe119la', 'safe119lo', 'safe119subtype'], 
    'parking' : [PARKING_MAX_RANGE, 'parkingid', 'parkingla', 'parkinglo', 'parkingtype'], 
}

## 함수 정의
- is_a_in_b
    - point는 (위도, 경도) 를 갖는 파이썬 튜플/리스트 객체 
    - a_point가 b_point 기준 +- (lo1m * max_range) 범위 안에 있는 경우 True 반환 
- get_dist_df
    - from_df의 물놀이장과 to_df 사이의 거리가 삽입된 dataframe을 반환 
    - 열 정보
        - 물놀이장의 일련번호 
        - 물놀이장의 이름 
        - 편의시설의 id
        - 편의시설의 type
        - 편의시설의 subtype
        - 물놀이장과 편의시설 사이의 거리 

In [21]:
def is_a_in_b(a_point, b_point, lo1m, la1m, max_range):
    LO_RANGE = b_point[0] - lo1m * max_range, b_point[0] + lo1m * max_range
    LA_RANGE = b_point[1] - la1m * max_range, b_point[1] + la1m * max_range
    
    # point : (위도, 경도)
    if (a_point[0] > LO_RANGE[0]) and (a_point[0] < LO_RANGE[1]):
        if (a_point[1] > LA_RANGE[0]) and (a_point[1] < LA_RANGE[1]):
            return True
    return False

In [22]:
def get_dist_df(from_df, to_df, faci_name):
    wpid = 0
    MAX_RANGE = faci_dict[faci_name][0]
    
    dist_df = pd.DataFrame(columns = [
        'distwpnumber',
        'distwpname',
        'distfaciid',
        'distfacitype',
        'distfacisubtype',
        'distdist',
    ])
    
    for i in range(len(from_df)):
        from_row = from_df.iloc[i]
        from_name, from_point = from_row['waterplayname'], (from_row['waterplayla'], from_row['waterplaylo'])
        
        for j in range(len(to_df)):
            to_row = to_df.iloc[j]
            to_id, to_la, to_lo, to_type = to_row[faci_dict[faci_name][1:]]
            to_point = to_la, to_lo
            
            if is_a_in_b(from_point, to_point, LO1M, LA1M, MAX_RANGE):
                dist_ab = haversine(from_point, to_point, unit = 'm')
                
                if dist_ab < MAX_RANGE:
                    dist_df.loc[wpid] = [i, from_name, to_id, faci_name, to_type, int(dist_ab)]
                    wpid += 1
        
    print("sucess ", faci_name)
                    
    return dist_df

In [23]:
# test
# get_dist_df(wp_df, safe119_df, 'safe119')

## 화장실 거리 결정
- 200m 또는 300m
- 분포를 보고 결정

In [32]:
# 화장실 단위거리가 300m인 경우
TOILET_MAX_RANGE = 300
faci_dict = {
    'toilet' : [TOILET_MAX_RANGE, 'toiletid', 'toiletla', 'toiletlo', 'toilettype'], 
}
_distdf_toilet_300 = get_dist_df(wp_df, toilet_df, 'toilet')

# 화장실 단위거리가 200m인 경우
TOILET_MAX_RANGE = 200
faci_dict = {
    'toilet' : [TOILET_MAX_RANGE, 'toiletid', 'toiletla', 'toiletlo', 'toilettype'], 
    'conv' : [CONV_MAX_RANGE, 'convid', 'convla', 'convlo', 'convtype'], 
    'drug' : [DRUG_MAX_RANGE, 'convid', 'convla', 'convlo', 'convtype'], 
    'medi' : [MEDI_MAX_RANGE, 'mediid', 'medila', 'medilo', 'meditype'], 
    'safe112' : [SAFE112_MAX_RANGE, 'safe112id', 'safe112la', 'safe112lo', 'safe112subtype'], 
    'safe119' : [SAFE119_MAX_RANGE, 'safe119id', 'safe119la', 'safe119lo', 'safe119subtype'], 
    'parking' : [PARKING_MAX_RANGE, 'parkingid', 'parkingla', 'parkinglo', 'parkingtype'], 
}
_distdf_toilet_200 = get_dist_df(wp_df, toilet_df, 'toilet')

sucess  toilet


In [35]:
len(_distdf_toilet_300)
_distdf_toilet_300.head()
len(_distdf_toilet_200)
_distdf_toilet_200.head()

404

,distwpnumber,distwpname,distfaciid,distfacitype,distfacisubtype,distdist
0,0,서울식물원 물놀이터,KCKIFPO22N000002472,toilet,공중화장실,0
1,0,서울식물원 물놀이터,KCKIFPO22N000002473,toilet,공중화장실,0
2,0,서울식물원 물놀이터,KCKIFPO22N000002474,toilet,공중화장실,0
3,0,서울식물원 물놀이터,KCKIFPO22N000002475,toilet,공중화장실,0
4,0,서울식물원 물놀이터,KCKIFPO22N000002476,toilet,공중화장실,0


211

,distwpnumber,distwpname,distfaciid,distfacitype,distfacisubtype,distdist
0,0,서울식물원 물놀이터,KCKIFPO22N000002472,toilet,공중화장실,0
1,0,서울식물원 물놀이터,KCKIFPO22N000002473,toilet,공중화장실,0
2,0,서울식물원 물놀이터,KCKIFPO22N000002474,toilet,공중화장실,0
3,0,서울식물원 물놀이터,KCKIFPO22N000002475,toilet,공중화장실,0
4,0,서울식물원 물놀이터,KCKIFPO22N000002476,toilet,공중화장실,0


In [38]:
_distdf_toilet_300['distwpnumber'].value_counts().sort_index()
_distdf_toilet_200['distwpnumber'].value_counts().sort_index()
# 단위거리가 300m인 경우
# 140개의 물놀이장 중 110개의 물놀이장 근처에 화장실이 단위거리 내에 있음
# 단위거리가 200m인 경우
# 140개의 물놀이장 중 87개의 물놀이장 근처에 화장실이 단위거리 내에 있음
# 따라서 화장실의 단위거리는 300m로 설정
# 근거는 공원의 크기와 거리 데이터의 분포임

0      10
1       1
3       6
4       3
5       9
       ..
134     1
135     5
136     1
137     2
138     1
Name: distwpnumber, Length: 110, dtype: int64

0      9
1      1
3      3
4      2
5      6
      ..
129    1
130    1
132    1
134    1
135    3
Name: distwpnumber, Length: 87, dtype: int64

## dist.csv 생성

In [40]:
TOILET_MAX_RANGE = 300
faci_dict = {
    'toilet' : [TOILET_MAX_RANGE, 'toiletid', 'toiletla', 'toiletlo', 'toilettype'], 
    'conv' : [CONV_MAX_RANGE, 'convid', 'convla', 'convlo', 'convtype'], 
    'drug' : [DRUG_MAX_RANGE, 'convid', 'convla', 'convlo', 'convtype'], 
    'medi' : [MEDI_MAX_RANGE, 'mediid', 'medila', 'medilo', 'meditype'], 
    'safe112' : [SAFE112_MAX_RANGE, 'safe112id', 'safe112la', 'safe112lo', 'safe112subtype'], 
    'safe119' : [SAFE119_MAX_RANGE, 'safe119id', 'safe119la', 'safe119lo', 'safe119subtype'], 
    'parking' : [PARKING_MAX_RANGE, 'parkingid', 'parkingla', 'parkinglo', 'parkingtype'], 
}

In [41]:
conv_df_conv = conv_df[conv_df['convtype'] == '편의점']
conv_df_drug = conv_df[conv_df['convtype'] != '편의점']

In [42]:
result_df = pd.concat([
#     get_dist_df(wp_df, toilet_df, 'toilet'),
    _distdf_toilet_300,
    get_dist_df(wp_df, conv_df_conv, 'conv'),
    get_dist_df(wp_df, conv_df_drug, 'drug'),
    get_dist_df(wp_df, medi_df, 'medi'),
    get_dist_df(wp_df, safe112_df, 'safe112'),
    get_dist_df(wp_df, safe119_df, 'safe119'),
    get_dist_df(wp_df, parking_df, 'parking'),
], ignore_index=True)

sucess  conv
sucess  drug
sucess  medi
sucess  safe112
sucess  safe119
sucess  parking


In [43]:
result_df

,distwpnumber,distwpname,distfaciid,distfacitype,distfacisubtype,distdist
0,0,서울식물원 물놀이터,KCKIFPO22N000002472,toilet,공중화장실,0
1,0,서울식물원 물놀이터,KCKIFPO22N000002473,toilet,공중화장실,0
2,0,서울식물원 물놀이터,KCKIFPO22N000002474,toilet,공중화장실,0
3,0,서울식물원 물놀이터,KCKIFPO22N000002475,toilet,공중화장실,0
4,0,서울식물원 물놀이터,KCKIFPO22N000002476,toilet,공중화장실,0
...,...,...,...,...,...,...
22301,134,솔길어린이공원 물놀이장,KC490PC22N013244,parking,공영,482
22302,135,동작주차공원,KC490PC22N004183,parking,공영,91
22303,135,동작주차공원,KC490PC22N006174,parking,공영,317
22304,135,동작주차공원,KC490PC22N006175,parking,공영,317


In [46]:
# 길이 확인
sum(result_df['distfacitype'] == 'toilet')
sum(result_df['distfacitype'] == 'conv')
sum(result_df['distfacitype'] == 'medi')
sum(result_df['distfacitype'] == 'safe112')
sum(result_df['distfacitype'] == 'safe119')
sum(result_df['distfacitype'] == 'parking')

404

2791

17881

383

362

317

In [47]:
# dataframe 저장
result_df.to_csv('../../data/물놀이/dist.csv', index=False, encoding='utf-8')
result_df.to_csv('../../data/물놀이/dist_euc.csv', index=False, encoding='euc-kr')